# Molecular Dynamics of Docked Structures


You'll need...

```bash
conda install -c conda-forge pdbfixer openmm
```

In [ ]:
## Load Libraries
from openmm.app import *
from openmm import *
from openmm.unit import *
from openmm.app import PDBFile
from pdbfixer import PDBFixer

from sys import stdout
import os


In [ ]:
input_pbd_file = '../HADDOCK Output Structures/6m0j/6M0J_ACE2_BA286.pdb'
output_pdb_file = input_pbd_file.replace('../HADDOCK Output Structures/', './').replace('.pdb', '_clean.pdb')

print('Cleaning up PDB file...')
print(f'\tInput PDB File: {input_pbd_file}')

## Make folders for output file
output_folder = os.path.dirname(output_pdb_file)
if not os.path.exists(output_folder):
    os.makedirs(output_folder)



In [ ]:

print('Fixing PDB file...')

fixer = PDBFixer(filename=input_pbd_file)
fixer.findMissingResidues()
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()
fixer.removeHeterogens(True)
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(7.0)
# fixer.addSolvent(fixer.topology.getUnitCellDimensions())
PDBFile.writeFile(fixer.topology, fixer.positions, open(output_pdb_file, 'w'))

print(f'\tOutput PDB File: {output_pdb_file}')

In [ ]:
pdb = PDBFile(output_pdb_file)
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

modeller = Modeller(pdb.topology, pdb.positions)
modeller.deleteWater()

residues = modeller.addHydrogens(forcefield)
modeller.addSolvent(forcefield, padding=1.0*nanometer)

print('Creating OpenMM System...')
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME, nonbondedCutoff=1.0*nanometer, constraints=HBonds)
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)

In [11]:
print("Minimizing energy...")
simulation.minimizeEnergy(maxIterations = 1)
## In theory, we're already at the lowest (or low-ish) energy from HADDOCK?

In [ ]:
simulation.reporters.append(PDBReporter(output_pdb_file, 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True, volume=True))

output_pdb_md_log_file = output_pdb_file.replace('.pdb', '_log.txt')
simulation.reporters.append(StateDataReporter(output_pdb_md_log_file,
                                              100,
                                              step=True,
                                              potentialEnergy=True,
                                              temperature=True,
                                              volume=True))

In [ ]:
print("Running NVT...")
simulation.step(10000)

In [ ]:
# system.addForce(MonteCarloBarostat(1*bar, 300*kelvin))
# simulation.context.reinitialize(preserveState=True)

# print("Running NPT...")
# simulation.step(10000)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
data = np.loadtxt(output_pdb_md_log_file, delimiter=',')

step = data[:,0]
potential_energy = data[:,1]
temperature = data[:,2]
volume = data[:,3]

plt.plot(step, potential_energy)
plt.xlabel("Step")
plt.ylabel("Potential energy (kJ/mol)")
plt.show()
plt.plot(step, temperature)
plt.xlabel("Step")
plt.ylabel("Temperature (K)")
plt.show()
plt.plot(step, volume)
plt.xlabel("Step")
plt.ylabel("Volume (nm^3)")
plt.show()